In [0]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten, BatchNormalization
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
import numpy as np

In [0]:
x_train = np.load("train_X.npy")
y_train = np.load("train_y.npy")
val_X = np.load("val_X.npy")
val_Y = np.load("val_y.npy")
test_X = np.load("public_test_X.npy")
test_Y = np.load("public_test_y.npy")
private_test_X = np.load("private_test_X.npy")
train_norm = x_train.astype('float32')
test_norm = test_X.astype('float32')
val_norm = val_X.astype('float32')
val_y = val_Y.astype('float32')
private_test_X = private_test_X.astype('float32')
# normalize to range 0-1
private_test_norm = private_test_X / 255.0
train_norm = train_norm / 255.0
test_norm = test_norm / 255.0
val_norm_x = val_norm / 255.0
one_hot_val_y = keras.utils.to_categorical(val_y)
private_test = private_test_X / 255.0
one_hot_test_Y = keras.utils.to_categorical(test_Y)
one_hot_train_Y = keras.utils.to_categorical(y_train)

In [0]:
print(num_classes)

10


In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3),kernel_initializer='he_uniform', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Conv2D(128, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

In [0]:
# Compile model
epochs = 30
lrate = 0.0001
#decay = lrate/epochs
#sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
model.summary()
# Fit the model
model.fit(train_norm, one_hot_train_Y, validation_data=(val_norm_x, one_hot_val_y), epochs=epochs, batch_size=128)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_61 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
dropout_66 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 16, 16, 64)      

In [0]:
model.fit(train_norm, one_hot_train_Y, validation_data=(val_norm_x, one_hot_val_y), epochs=epochs, batch_size=128)

Train on 47000 samples, validate on 1000 samples
Epoch 1/30
47000/47000 [==============================] - 7s 159us/step - loss: 0.1297 - acc: 0.9579 - val_loss: 0.5604 - val_acc: 0.8440
Epoch 2/30
47000/47000 [==============================] - 7s 159us/step - loss: 0.1303 - acc: 0.9573 - val_loss: 0.5827 - val_acc: 0.8600
Epoch 3/30
47000/47000 [==============================] - 8s 161us/step - loss: 0.1269 - acc: 0.9584 - val_loss: 0.5902 - val_acc: 0.8480
Epoch 4/30
47000/47000 [==============================] - 8s 161us/step - loss: 0.1196 - acc: 0.9607 - val_loss: 0.6892 - val_acc: 0.8250
Epoch 5/30
47000/47000 [==============================] - 7s 158us/step - loss: 0.1275 - acc: 0.9591 - val_loss: 0.6662 - val_acc: 0.8330
Epoch 6/30
47000/47000 [==============================] - 7s 159us/step - loss: 0.1225 - acc: 0.9603 - val_loss: 0.5966 - val_acc: 0.8620
Epoch 7/30
47000/47000 [==============================] - 8s 162us/step - loss: 0.1189 - acc: 0.9619 - val_loss: 0.6232 - v

In [0]:
x = model.predict_classes(private_test_norm)
def save_csv(x, filename="submission.csv"):
    """save_csv Save the input into csv file

    Arguments:
        x {np.ndarray} -- input array

    Keyword Arguments:
        filename {str} -- The file name (default: {"submission.csv"})

    Raises:
        ValueError: Input data structure is not np.ndarray
    """
    if isinstance(x, np.ndarray):
        x = x.flatten()
        np.savetxt(filename, x, delimiter=',',fmt='%i')
    else:
        raise ValueError("The input is not an np.ndarray")
save_csv(x)

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten, BatchNormalization
from keras.constraints import maxnorm
from keras.utils import to_categorical
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
import numpy as np


Using TensorFlow backend.


In [0]:
x_train = np.load("train_X.npy").astype('float32')/255.0
y_train = np.load("train_y.npy").astype('float32')
onehot_train_Y = to_categorical(y_train)
val_X = np.load("val_X.npy").astype('float32')/255.0
val_Y = np.load("val_y.npy").astype('float32')
onehot_val_Y = to_categorical(val_Y)
private_test_X = np.load("private_test_X.npy").astype('float32')/255.0

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3),kernel_initializer='he_uniform', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Conv2D(128, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu',kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [0]:
epochs = 30
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
# Fit the model
model.fit(x_train, onehot_train_Y, validation_data=(val_X, onehot_val_Y), epochs=epochs, batch_size=128)



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)     